<a href="https://colab.research.google.com/github/serawit/serawit/blob/main/Serawit_Auto_Coffe_LDisisDetI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# automatic coffee disease and pest damage identification system using recent image processing techniques.

Mounting and connecting to google drive (to access the image dataset)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Library installing & importing

In [ ]:
!pip install -q opencv-python
!pip install -q numpy
!pip install -q scikit-learn
!pip install -q matplotlib

Importing Libraries

In [ ]:
import os
import numpy as np
import cv2 as cv
import sklearn.datasets as datasets
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

Load & display:-
*Loads and displays only the unique ones*

In [ ]:
# Specify the path to the directory containing your images
image_dir_path = '/content/drive/MyDrive/rust'

In [ ]:
# Specify the path to the directory containing your images
image_dir_path = '/content/drive/MyDrive/healthy_images/Healthy'

In [ ]:

# Set to store unique labels encountered
seen_labels = set()

# Function to process images in a directory
def process_images(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        if os.path.isdir(file_path):  # Check if it's a directory
            process_images(file_path)  # Recursively process subdirectories
        elif filename.lower().endswith(('.jpg', '.jpeg', '.png')):  # Check if it's an image file
            img = cv.imread(file_path)
            if img is not None:
                label = filename.split(' ')[0].strip()  # Extract the label
                if label not in seen_labels:  # Check if label is unique
                    seen_labels.add(label)  # Add the label to the set of seen labels
                    img_rgb = cv.cvtColor(img, cv.COLOR_BGR2RGB)
                    plt.figure(figsize=(6, 6))
                    plt.subplot(222)
                    plt.title(f'Label: {label}')
                    plt.imshow(img_rgb)
                    plt.axis('off')
                    plt.show()
            else:
                print(f"Failed to load image: {file_path}")


In [ ]:
# Start processing images from the main directory
process_images(image_dir_path)



Feature extraction

Grayscale feature

In [ ]:
def extract_grayscale_features(image_path):
    image = cv.imread(image_path, cv.IMREAD_GRAYSCALE)
    if image is not None:
        return image.flatten()
    else:
        print(f"Failed to load image: {image_path}")
        return None

In [ ]:
def process_images(image_dir_path):
    all_features = []
    all_images = [] # Store the grayscale images
    for filename in os.listdir(image_dir_path):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(image_dir_path, filename)
            image = cv.imread(image_path, cv.IMREAD_GRAYSCALE)
            if image is not None:
                all_features.append(image.flatten())
                all_images.append(image) # Store the image

    return all_features, all_images

In [ ]:
# Execute/call
image_dir_path = '/content/drive/MyDrive/healthy_images/Healthy'
all_image_features = process_images(image_dir_path)

# Now you have a list of flattened features, one for each image
print(all_image_features)

In [ ]:
def visualize_features(features, original_image_path):
    # Load original image in color
    original_image = cv.imread(original_image_path)

    # Reshape the flattened features
    reshaped_image = features.reshape(original_image.shape[:2])  # Use only height and width for reshaping

    # Display side by side
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    axes[0].imshow(cv.cvtColor(original_image, cv.COLOR_BGR2RGB))  # Convert BGR to RGB for matplotlib
    axes[0].set_title('Original Image')
    axes[1].imshow(reshaped_image, cmap='gray')
    axes[1].set_title('Visualized Features')
    plt.show()

In [ ]:
# Visualize features for the first image (adjust index as needed)
if all_image_features:
    first_image_path = os.path.join(image_dir_path, os.listdir(image_dir_path)[0])
    visualize_features(all_image_features[0], first_image_path)

**Edge feature extraction**

In [ ]:
def extract_canny_edges(gray_image, low_threshold=50, high_threshold=150):
    edges = cv.Canny(gray_image, low_threshold, high_threshold)
    return edges.flatten()

In [ ]:
def process_canny_images(image_dir_path):
    all_edge_features = []
    for filename in os.listdir(image_dir_path):
        if filename.endswith(('.jpg', '.jpeg', '.png')):
            image_path = os.path.join(image_dir_path, filename)
            edge_features = extract_canny_edges(image_path)
            if edge_features is not None:
                all_edge_features.append(edge_features)

    return all_edge_features

In [ ]:
# Process images for Canny edge features
#all_edge_features = process_canny_images(image_dir_path)

In [ ]:
# Process images for grayscale features and store grayscale images
all_image_features, all_images = process_images(image_dir_path)

# Process grayscale images for Canny edge features
all_edge_features = [extract_canny_edges(img) for img in all_images]

In [ ]:
def visualize_images(image_dir_path, all_images):
    # Get the first grayscale image
    if all_images:
        gray_image = all_images[0]

        # Load original color image (we still need to load this separately)
        first_image_path = os.path.join(image_dir_path, os.listdir(image_dir_path)[0])
        original_image = cv.imread(first_image_path)

        # Extract Canny edges
        edges = extract_canny_edges(gray_image)

        # Reshape the flattened edges array (assuming you know the original dimensions)
        original_height, original_width = gray_image.shape  # Replace with actual dimensions if known
        edges_reshaped = edges.reshape((original_height, original_width))

        # Display side by side
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))
        axes[0].imshow(cv.cvtColor(original_image, cv.COLOR_BGR2RGB))
        axes[0].set_title('Original Image')
        axes[1].imshow(gray_image, cmap='gray')
        axes[1].set_title('Grayscale Image')
        axes[2].imshow(edges_reshaped, cmap='gray') #use reshaped edges
        axes[2].set_title('Canny Edges')
        plt.show()
    else:
        print("No images found in the directory.")

In [ ]:
# Process images for grayscale features and store grayscale images
#all_image_features, all_images = process_images(image_dir_path)

# Visualize the first image, its grayscale (reused), and edges
visualize_images(image_dir_path, all_images)